In [1]:
import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import sqlite3

In [2]:
# Display non-truncated info in dataframes
pd.set_option('display.max_colwidth', -1)

## Data import

In [3]:
# Data exported from: https://www.acleddata.com/data/

df = pd.read_csv('../data/1997-01-01-2019-11-06.csv')
df.shape

(621082, 31)

### Basic exploration of dataset

In [6]:
# df.info()

In [7]:
# df.iloc[0]

In [8]:
df.iloc[0][['country', 'notes', 'event_date', 'source', 'event_type', 'sub_event_type', 'region', 'fatalities']]

country           Cambodia                                                                                                                                                                                                                                                                                                                        
notes             As of 2 November 2019, at least 49 CNRP activists have been arrested in an ongoing crackdown by the Cambodian government. Arrests have been made in Phnom Penh, Chbar Mon (Kampong Speu), Kampong Cham, Paoy Paet (Banteay Meanchey), Batheay (Kampong Cham), Banlung (Ratanak Kiri), Kandal, Kampot and Keo Siema (Mondulkiri).
event_date        02 November 2019                                                                                                                                                                                                                                                                                                

In [9]:
df.year.dtypes

dtype('int64')

In [10]:
# Filter: Count 'strategic developments' incidents documented in Cambodia in 2019
len(df[(df.event_type == 'Strategic developments') & (df.year == 2019) & (df.country == 'Cambodia')])

8

In [11]:
# Number of countries in database in 2019
countries_2019 = df[df.year == 2019]
countries_2019.country.nunique()

100

## Format database

In [12]:
columns = ['data_id', 'event_date',\
           'year', 'event_type', 'notes',\
           'source', 'country','sub_event_type', 'region', 'fatalities' ]

df = df[columns]
df.shape

(621082, 10)

In [13]:
# New column for date
df['date'] = pd.to_datetime(df['event_date'])

In [14]:
# Year column format
df['year'] = df['date'].dt.year

In [15]:
df.year

0         2019
1         2019
2         2019
3         2019
4         2019
5         2019
6         2019
7         2019
8         2019
9         2019
10        2019
11        2019
12        2019
13        2019
14        2019
15        2019
16        2019
17        2019
18        2019
19        2019
20        2019
21        2019
22        2019
23        2019
24        2019
25        2019
26        2019
27        2019
28        2019
29        2019
          ... 
621052    1997
621053    1997
621054    1997
621055    1997
621056    1997
621057    1997
621058    1997
621059    1997
621060    1997
621061    1997
621062    1997
621063    1997
621064    1997
621065    1997
621066    1997
621067    1997
621068    1997
621069    1997
621070    1997
621071    1997
621072    1997
621073    1997
621074    1997
621075    1997
621076    1997
621077    1997
621078    1997
621079    1997
621080    1997
621081    1997
Name: year, Length: 621082, dtype: int64

In [16]:
# Set index, sort
df = df.set_index(['date'])
df.sort_values('date', inplace=True)

In [17]:
df.head()

,data_id,event_date,year,event_type,notes,source,country,sub_event_type,region,fatalities
date,,,,,,,,,,
1997-01-01,6284959,01 January 1997,1997,Battles,Security forces kill a rebel officer who had been disrupting security in the region of Cyangugu.,Local Source,Rwanda,Armed clash,Eastern Africa,1
1997-01-01,5760768,01 January 1997,1997,Violence against civilians,The total number of chiefs in the past 6 months assassinated in this province=10,Diario de Noticias (Angola),Angola,Attack,Middle Africa,10
1997-01-01,5016294,01 January 1997,1997,Violence against civilians,30 cadavers found October 14,Angola Cronologia,Angola,Attack,Middle Africa,30
1997-01-01,5016300,01 January 1997,1997,Battles,"4 deaths reported, denied by both sides, anti-patasse side supported by moise kette of chad",All Africa,Central African Republic,Armed clash,Middle Africa,2
1997-01-01,5768186,01 January 1997,1997,Battles,Fighting between NDA rebels and Sudanese government forces is underway after an attack last week,Inter Press Service,Sudan,Armed clash,Northern Africa,0


In [18]:
#Filter range
df_since_2010 = df.loc['2010-01-01':'2019-11-02']

In [19]:
df_since_2010.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 570871 entries, 2010-01-01 to 2019-11-02
Data columns (total 10 columns):
data_id           570871 non-null int64
event_date        570871 non-null object
year              570871 non-null int64
event_type        570871 non-null object
notes             570851 non-null object
source            570871 non-null object
country           570871 non-null object
sub_event_type    570871 non-null object
region            570871 non-null object
fatalities        570871 non-null int64
dtypes: int64(3), object(7)
memory usage: 47.9+ MB


## Create sqlite db

In [20]:
path_to_db = os.path.abspath('../data/working_data')

try: 
    os.mkdir(path_to_db)
except:
    pass
    print('Error')

Error


In [21]:
# Create and connect to db
conn = sqlite3.connect('../data/working_data/working-database.db')
conn

In [22]:
# upload df_since_2010 to database
%time df_since_2010.to_sql('all_incidents', con=conn)

CPU times: user 3.17 s, sys: 331 ms, total: 3.51 s
Wall time: 3.69 s


In [23]:
# get indexes
pd.read_sql_query("PRAGMA index_list(all_incidents);", conn)

,seq,name,unique,origin,partial
0,0,ix_all_incidents_date,0,c,0


In [24]:
# Time query data by country
%time search_query = pd.read_sql_query("SELECT * from all_incidents WHERE country LIKE 'Somalia';", conn)
search_query.head()

CPU times: user 170 ms, sys: 51.3 ms, total: 221 ms
Wall time: 222 ms


,date,data_id,event_date,year,event_type,notes,source,country,sub_event_type,region,fatalities
0,2010-01-01 00:00:00,6304109,01 January 2010,2010,Battles,Fighters loyal to Hisb Al-Islam group reportedly killed seven people in Toorotoorow District of Lower Shabeelle Region after clashing with local militia in the town. Sources say that most of those who died in the fighting were civilians living in the area of the fighting.,Radio Gaalkacyo,Somalia,Armed clash,Eastern Africa,7
1,2010-01-01 00:00:00,6309080,01 January 2010,2010,Explosions/Remote violence,At least 5 people have been killed and more others have been wounded in parts of the Somali capital Mogadishu after heavy shelling landed into several areas in the capital.,Shabelle Media Network,Somalia,Shelling/artillery/missile attack,Eastern Africa,2
2,2010-01-02 00:00:00,5764547,02 January 2010,2010,Strategic developments,"Property destruction: Al Shabaab forces in Merka set fire on three hundred sacks of maize and beans that they confiscated from a WFP owned warehouse in Merka. They claimed that the food were contaminated and not fit for human consumption. Retrieved www on 03/01/2010, http://www.hiiraan.com/news2/2010/jan/somali_extremists_scorch_food_aid.aspx O3/Al Shabaab WFP /(0/10)/ (00)/ Merka/Lower Shabelle/Somalia/ 20100104.298",Hiiraan (Somalia),Somalia,Looting/property destruction,Eastern Africa,0
3,2010-01-02 00:00:00,6267562,02 January 2010,2010,Battles,"Somalias Al Shabaab insurgent group has reportedly seized the control of central town of Dhusamareb, the capital of Galgadud region. Local residents report that dozens have been killed in the fighting.",Garowe Online,Somalia,Armed clash,Eastern Africa,12
4,2010-01-02 00:00:00,4966071,02 January 2010,2010,Battles,"At least three people have died and four others wounded in a heavy fighting between Hisb Al-Islam group and pro-government forces at Ceel-gaal area. Both sides are reported to have exchanged heavy weapons, mortars and cannons prompting pastoralists in the area to vacate from their residences.",All Africa,Somalia,Armed clash,Eastern Africa,3


In [25]:
#Optimize performace by creating an index on countries

c = conn.cursor()
%time c.execute("CREATE INDEX idx1 ON all_incidents(country)")
conn.commit()

CPU times: user 266 ms, sys: 89.5 ms, total: 355 ms
Wall time: 365 ms


In [27]:
%time search_query.head()

CPU times: user 233 µs, sys: 23 µs, total: 256 µs
Wall time: 247 µs


,date,data_id,event_date,year,event_type,notes,source,country,sub_event_type,region,fatalities
0,2010-01-01 00:00:00,6304109,01 January 2010,2010,Battles,Fighters loyal to Hisb Al-Islam group reportedly killed seven people in Toorotoorow District of Lower Shabeelle Region after clashing with local militia in the town. Sources say that most of those who died in the fighting were civilians living in the area of the fighting.,Radio Gaalkacyo,Somalia,Armed clash,Eastern Africa,7
1,2010-01-01 00:00:00,6309080,01 January 2010,2010,Explosions/Remote violence,At least 5 people have been killed and more others have been wounded in parts of the Somali capital Mogadishu after heavy shelling landed into several areas in the capital.,Shabelle Media Network,Somalia,Shelling/artillery/missile attack,Eastern Africa,2
2,2010-01-02 00:00:00,5764547,02 January 2010,2010,Strategic developments,"Property destruction: Al Shabaab forces in Merka set fire on three hundred sacks of maize and beans that they confiscated from a WFP owned warehouse in Merka. They claimed that the food were contaminated and not fit for human consumption. Retrieved www on 03/01/2010, http://www.hiiraan.com/news2/2010/jan/somali_extremists_scorch_food_aid.aspx O3/Al Shabaab WFP /(0/10)/ (00)/ Merka/Lower Shabelle/Somalia/ 20100104.298",Hiiraan (Somalia),Somalia,Looting/property destruction,Eastern Africa,0
3,2010-01-02 00:00:00,6267562,02 January 2010,2010,Battles,"Somalias Al Shabaab insurgent group has reportedly seized the control of central town of Dhusamareb, the capital of Galgadud region. Local residents report that dozens have been killed in the fighting.",Garowe Online,Somalia,Armed clash,Eastern Africa,12
4,2010-01-02 00:00:00,4966071,02 January 2010,2010,Battles,"At least three people have died and four others wounded in a heavy fighting between Hisb Al-Islam group and pro-government forces at Ceel-gaal area. Both sides are reported to have exchanged heavy weapons, mortars and cannons prompting pastoralists in the area to vacate from their residences.",All Africa,Somalia,Armed clash,Eastern Africa,3
